In this notebook we cover how to triangulate the 3D locations of animals using [anipose](https://anipose.readthedocs.io/en/latest/index.html) and [SLEAP](https://sleap.ai/). <br>
We start with the tracked 2D locations of the animals, stored in separate hdf5 files for each camera view, along with the videos of the calibration board. <br>
We calibrate the camera parameters using th| calibration board videos, and then use those parameters in conjunction with our tracked 2D locations to derive the 3D points. 

# Imports

In [1]:
# Generic imports 

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import h5py 

In [2]:
# SLEAP 
import importlib.util

spec = importlib.util.find_spec('sleap')
if spec is None:
    #print(package_name +" is not installed")
    !pip install sleap

In [3]:
import sleap
sleap.versions()

SLEAP: 1.2.7
TensorFlow: 2.8.3
Numpy: 1.22.4
Python: 3.8.10
OS: Windows-10-10.0.19044-SP0


In [4]:
# Aniposelib 
spec = importlib.util.find_spec('aniposelib')
if spec is None:
    #print(package_name +" is not installed")
    !python -m pip install --user aniposelib

In [5]:
from aniposelib.boards import CharucoBoard, Checkerboard
from aniposelib.cameras import Camera, CameraGroup
from aniposelib.utils import load_pose2d_fnames

In [6]:
import preprocessing as pre

# Loading Data

In [7]:
analysis_files = ['C:\\Users\shantanu.ray\Downloads\labels.v001.005_CFL12_04072022_saline_27_14-39-24_cam1.analysis.h5',
                  'C:\\Users\shantanu.ray\Downloads\labels.v001.001_CFL12_04072022_saline_27_14-39-24_cam2.analysis.h5',
                  'C:\\Users\shantanu.ray\Downloads\labels.v001.000_CFL12_04072022_saline_27_14-39-24_cam3.analysis.h5']

In [8]:
SAMPLE_START = 100
SAMPLE_END  = 500

In [9]:
gt = [pre.get_2d_poses(file) for file in analysis_files]
gt = np.stack(gt, axis=0)[:, SAMPLE_START:SAMPLE_END]

n_cams, n_frames, n_nodes, _, n_tracks = gt.shape
gt.shape # (n_cams, n_sampled_frames, n_keypoints, 2, # tracks)

Tracks of shape: (1600, 11, 2, 1)

Tracks of shape: (1600, 11, 2, 1)

Tracks of shape: (1600, 11, 2, 1)



(3, 400, 11, 2, 1)

The get_2D_poses function returns 2D tracks for a single file at a time, so we append all the tracks to a list and then stack the 2D tracks on top of each other. 

# Calibration

Some of the following code was adapted from the [aniposelib tutorial](https://anipose.readthedocs.io/en/latest/aniposelib-tutorial.html#). 

In [13]:
board_vids = [['D:/Ayesha_local/RP_calibration/video1/cam1/calibration_video1_1_11-34-20_cam1.mp4'], # Insert your own video paths here 
        ['D:/Ayesha_local/RP_calibration/video1/cam2/calibration_video1_1_11-34-20_cam2.mp4'],
        ['D:/Ayesha_local/RP_calibration/video1/cam3/calibration_video1_1_11-34-20_cam3.mp4']]

cam_names = ['cam1', 'cam2', 'cam3']

board = CharucoBoard(5, 5, # width x height
                     square_length=5, # here, in cm but any unit works
                     marker_length=3.75,
                     marker_bits=4, dict_size=100)

cgroup = CameraGroup.from_names(cam_names)

cgroup.calibrate_videos(board_vids, board)
cgroup.dump('calibration.toml')

D:/Ayesha_local/RP_calibration/video1/cam1/calibration_video1_1_11-34-20_cam1.mp4


100%|██████████████████████████| 17781/17781 [01:18<00:00, 226.83it/s]


9609 boards detected
D:/Ayesha_local/RP_calibration/video1/cam2/calibration_video1_1_11-34-20_cam2.mp4


100%|██████████████████████████| 17781/17781 [01:18<00:00, 226.86it/s]


8577 boards detected
D:/Ayesha_local/RP_calibration/video1/cam3/calibration_video1_1_11-34-20_cam3.mp4


100%|██████████████████████████| 17781/17781 [00:40<00:00, 438.64it/s]


3083 boards detected
defaultdict(<class 'int'>,
            {('cam1', 'cam2'): 6893,
             ('cam1', 'cam3'): 553,
             ('cam2', 'cam1'): 6893,
             ('cam2', 'cam3'): 59,
             ('cam3', 'cam1'): 553,
             ('cam3', 'cam2'): 59})
INFO:numba.core.transforms:finding looplift candidates
error:  0.9958885662113552
n_samples: 100
{(0, 1): (1000, array([0.44847118, 1.60491573])),
 (0, 2): (1000, array([0.61815382, 2.60735789])),
 (1, 2): (253, array([1.48541073, 7.57382033]))}
error: 1.01, mu: 7.6, ratio: 0.942
INFO:numba.core.transforms:finding looplift candidates
INFO:numba.core.transforms:finding looplift candidates
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         8.8414e+03                                    5.57e+05    
       1              2         1.1401e+02      8.73e+03       7.91e+01       1.70e+04    
       2              4         2.9941e+01      8.41e+01       3.02e+01

The cgroup object refers to all the cameras in a single entity. It will keep track of the camera parameters for when we later need them for triangulation

In [10]:
cgroup = CameraGroup.load("C:\\Users\\shantanu.ray\\projects\\sleap\\addons\\calibration.toml")

# Triangulation

Aniposelib gives us the option to triangulate with the direct linear transformation (DLT) or with RANSAC, which adds an outlier rejection subroutine to the DLT. <br>
In addition to these 2 triangulation methods, we can further refine the 3D points via direct optimization of the reprojection error. 

In [21]:
# Original simple triangulation
p3d_dlt = np.stack([cgroup.triangulate_optim(gt[..., track], init_progress=True) for track in range(n_tracks)], axis=-1)
p3d_dlt.shape

100%|██████████████████████████| 4400/4400 [00:00<00:00, 10080.50it/s]


(400, 11, 3, 1)

In [40]:
# Ransac triangulation 
p3d_ransac = np.stack([cgroup.triangulate_optim(gt[..., track], init_ransac = True, init_progress = True) for track in range(n_tracks)], axis=-1)
p3d_ransac.shape

  0%|                                        | 0/4400 [00:00<?, ?it/s]

INFO:numba.core.transforms:finding looplift candidates


100%|████████████████████████████| 4400/4400 [00:08<00:00, 499.06it/s]
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


INFO:numba.core.transforms:finding looplift candidates


(400, 11, 3, 1)

In [22]:
# Refinement
p3d_ls = np.stack([cgroup.optim_points(gt[..., track], p3d_dlt[..., track]) for track in range(n_tracks)], axis=-1) 
p3d_ls.shape

(400, 11, 3, 1)

In [18]:
np.save('p3d_ls.npy', p3d_ls)

In [ ]:
from scipy.io import savemat

In [11]:
from triangulation import TriangulationMethod, get_3d_poses

In [12]:
p3d_dlt_1 = get_3d_poses(poses_2d=gt,
                         calibration_filepath="C:\\Users\\shantanu.ray\\projects\\sleap\\addons\\calibration.toml",
                         triangulate=TriangulationMethod.calibrated_dtl,
                         show_progress=True)

100%|███████████████████████████| 4400/4400 [00:03<00:00, 1365.92it/s]
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


INFO:numba.core.transforms:finding looplift candidates
INFO:numba.core.transforms:finding looplift candidates


In [13]:
p3d_dlt_1.shape

(400, 11, 3, 1)

In [14]:
p3d_ransac_1 = get_3d_poses(poses_2d=gt,
                         calibration_filepath="C:\\Users\\shantanu.ray\\projects\\sleap\\addons\\calibration.toml",
                         triangulate=TriangulationMethod.calibrated_ransac,
                         show_progress=True)

  0%|                                        | 0/4400 [00:00<?, ?it/s]

INFO:numba.core.transforms:finding looplift candidates


100%|████████████████████████████| 4400/4400 [00:06<00:00, 705.11it/s]
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


INFO:numba.core.transforms:finding looplift candidates


In [15]:
p3d_ransac_1.shape

(400, 11, 3, 1)

In [16]:
p3d_dlt_2 = get_3d_poses(poses_2d=gt,
                         calibration_filepath="C:\\Users\\shantanu.ray\\projects\\sleap\\addons\\calibration.toml",
                         triangulate=TriangulationMethod.calibrated_dtl,
                         refine_calibration=True,
                         show_progress=True)

100%|███████████████████████████| 4400/4400 [00:00<00:00, 9386.68it/s]
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\shantanu.ray\projects\sleap\sleap\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
p3d_dlt_2.shape

(400, 11, 3, 1)